In [12]:
# Load the dataset
import pandas as pd

data = pd.read_csv('/Volumes/Mac/DatasetSP/pipeline2/combined_features_2.csv')

In [14]:
data.head()
data.describe()

,Subject,Test,Trial,Bout,Position,Acc_X_Var,Acc_X_RMS,Acc_X_SMA,Acc_X_NumPeaks,Acc_X_DomFreq,...,Gyr_Z_DomFreq,Gyr_Z_DomPower,Gyr_Z_SpecEntropy,Gyr_Norm_Var,Gyr_Norm_RMS,Gyr_Norm_SMA,Gyr_Norm_NumPeaks,Gyr_Norm_DomFreq,Gyr_Norm_DomPower,Gyr_Norm_SpecEntropy
count,105209.000000,105209.000000,105209.000000,105209.000000,105209.000000,105209.000000,105209.000000,105209.000000,105209.000000,105209.000000,...,105209.000000,105209.000000,105209.000000,105209.000000,105209.000000,105209.000000,105209.000000,105209.000000,105209.000000,105209.000000
mean,10.955080,9.549117,3.552614,13.932734,2.452138,8.270679,7.995806,7.524544,21.557566,1.984675,...,1.539989,5194.642667,4.338527,2964.968677,97.222385,85.304637,24.618882,1.649868,5129.851613,4.618699
std,7.536324,1.196259,1.048692,11.981405,1.704596,9.782387,3.003762,2.954363,4.303126,1.189416,...,1.050549,5322.966559,0.294788,4421.555073,56.990357,49.580597,7.094725,1.187916,3756.954804,0.270135
min,1.000000,4.000000,1.000000,1.000000,0.000000,0.000023,0.070292,0.069966,5.000000,0.333333,...,0.333333,1.589329,2.482545,0.000776,0.072882,0.066181,3.000000,0.333333,1.365878,2.934905
25%,3.000000,10.000000,4.000000,3.000000,1.000000,2.431045,5.435751,4.951134,19.000000,1.666667,...,1.000000,2335.889622,4.179326,586.763612,52.791321,46.400970,19.000000,0.666667,2514.730552,4.474205
50%,9.000000,10.000000,4.000000,10.000000,2.000000,5.276473,9.544130,9.185090,22.000000,1.666667,...,1.000000,3635.712427,4.364282,1547.179659,84.457471,74.372315,25.000000,1.666667,4155.232317,4.664772
75%,17.000000,10.000000,4.000000,24.000000,4.000000,10.593992,10.063291,9.705929,25.000000,2.000000,...,2.000000,5840.479340,4.536658,3652.364281,130.769748,115.204345,30.000000,2.000000,6565.516793,4.810157
max,25.000000,10.000000,4.000000,72.000000,5.000000,516.422327,26.711397,17.730761,54.000000,20.000000,...,15.666667,56304.119592,5.673936,130617.824302,587.701373,508.672772,77.000000,14.000000,56790.240330,5.670880


In [15]:
costruction_set = data[data['Subject'].isin([8, 16, 1, 9, 6, 19, 20, 17, 4, 7, 25, 3])]
test_set = data[data['Subject'].isin([2, 15, 22])]

In [18]:
costruction_set.to_csv('/Volumes/Mac/DatasetSP/pipeline2/construction_set_2.csv', index=False)
test_set.to_csv('/Volumes/Mac/DatasetSP/pipeline2/test_set_2.2.csv', index=False)

In [17]:
# Installazione di MLflow se necessario
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.model_selection import GroupKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import time
import os
import pickle
from tqdm import tqdm
from xgboost import XGBClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

mlflow.set_tracking_uri("http://127.0.0.1:5000")  # Imposta il percorso per salvare i log
mlflow.set_experiment("Pipeline2.2")  # Nome dell'esperimento

print("Esperimento MLflow inizializzato!")

# Funzione per caricare il dataset
def load_data(file_path):
    data = pd.read_csv(file_path)
    return data

# Funzione per preprocessare i dati
def preprocess_data(data):
    X = data.drop(columns=['Position', 'Subject', 'Trial', 'Test', 'Bout'])
    y = data['Position']
    subjects = data['Subject']
    return X, y, subjects

# Percorso del dataset
data_file = "/Volumes/Mac/DatasetSP/pipeline2/construction_set_2.2.csv"

# Caricamento dati
print("Caricamento del dataset...")
data = load_data(data_file)
print("Dataset caricato con successo!")

print("Preprocessing dei dati...")
X, y, subjects = preprocess_data(data)
print("Preprocessing completato!")

# Definizione input per i modelli
input_dim = X.shape[1]
num_classes = len(np.unique(y))


Esperimento MLflow inizializzato!
Caricamento del dataset...
Dataset caricato con successo!
Preprocessing dei dati...
Preprocessing completato!


In [18]:
# Funzione per generare e salvare i fold
def save_folds(X, y, subjects, n_splits=5, save_path="fold_indices.pkl"):
    group_kfold = GroupKFold(n_splits=n_splits)
    folds = list(group_kfold.split(X, y, groups=subjects))
    with open(save_path, "wb") as f:
        pickle.dump(folds, f)
    print(f"Folds salvati in {save_path}")

# Funzione per caricare i fold
def load_folds(load_path="fold_indices.pkl"):
    try:
        with open(load_path, "rb") as f:
            folds = pickle.load(f)
        print(f"Folds caricati da {load_path}")
        return folds
    except FileNotFoundError:
        print(f"Errore: Il file {load_path} non esiste.")
        return None

# Creazione o caricamento dei fold
fold_path = "/Users/giorgio/Desktop/SmartphonePositioning/src/modeling/fold_indices_p2.2.pkl"
folds = load_folds(fold_path)
if folds is None:
    save_folds(X, y, subjects, save_path=fold_path)
    folds = load_folds(fold_path)


Errore: Il file /Users/giorgio/Desktop/SmartphonePositioning/src/modeling/fold_indices_p2.2.pkl non esiste.
Folds salvati in /Users/giorgio/Desktop/SmartphonePositioning/src/modeling/fold_indices_p2.2.pkl
Folds caricati da /Users/giorgio/Desktop/SmartphonePositioning/src/modeling/fold_indices_p2.2.pkl


In [19]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_true, y_pred, model_name, fold_idx):
    """Genera e salva la Confusion Matrix come immagine."""
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y_true), yticklabels=np.unique(y_true))
    plt.xlabel("Predetto")
    plt.ylabel("Reale")
    plt.title(f"Confusion Matrix - {model_name} (Fold {fold_idx})")

    # Salvataggio immagine
    cm_filename = f"{model_name}_confusion_matrix_fold_{fold_idx}.png"
    plt.savefig(cm_filename)
    plt.close()

    return cm_filename


In [20]:
import json
import pandas as pd

models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42),
    'XGBoost': XGBClassifier(n_jobs=8, max_depth=8, colsample_bytree=0.8, subsample=0.8, n_estimators=150, random_state=42),
    'Logistic Regression': LogisticRegression(n_jobs=-1, max_iter=1000, random_state=42)
}

def train_and_evaluate_mlflow(model_name, model, X, y, subjects, folds, experiment_name):
    print(f"\nInizio training modello: {model_name}")
    start_time = time.time()

    # Inizializza l'esperimento in MLflow
    mlflow.set_experiment(experiment_name)

    with mlflow.start_run(run_name=model_name):
        # Definizione dei percorsi per l'organizzazione dei file
        experiment_path = os.path.join("mlruns", experiment_name)
        model_path = os.path.join(experiment_path, "models", model_name)
        log_path = os.path.join(experiment_path, "logs", model_name)
        artifact_path = os.path.join(experiment_path, "artifacts", model_name)

        os.makedirs(model_path, exist_ok=True)
        os.makedirs(log_path, exist_ok=True)
        os.makedirs(artifact_path, exist_ok=True)

        # Log dei parametri del modello
        if hasattr(model, "get_params"):
            params = model.get_params()
            mlflow.log_params(params)  # Logga tutti i parametri del modello
        else:
            print(f"[{model_name}] Nessun parametro disponibile per il log.")

        cv_scores = []
        reports = []

        for fold_idx, (train_idx, test_idx) in enumerate(tqdm(folds, desc="Cross-validation folds")):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            score = accuracy_score(y_test, y_pred)
            cv_scores.append(score)

            # Generazione del classification report
            report = classification_report(y_test, y_pred, output_dict=True)
            reports.append(report)

            # Creazione della Confusion Matrix
            cm_filename = os.path.join(log_path, f"confusion_matrix_fold_{fold_idx}.png")
            cm = confusion_matrix(y_test, y_pred)
            plt.figure(figsize=(8, 6))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
            plt.xlabel('Predicted')
            plt.ylabel('Actual')
            plt.title(f'Confusion Matrix - {model_name} Fold {fold_idx}')
            plt.savefig(cm_filename)
            plt.close()

            # Log delle metriche su MLflow
            mlflow.log_metric(f"accuracy_fold_{fold_idx}", score)
            mlflow.log_artifact(cm_filename)  # Log della Confusion Matrix su MLflow

        # Log finale su MLflow
        mlflow.log_metric("Mean_CV_Accuracy", np.mean(cv_scores))
        mlflow.log_metric("Std_CV_Accuracy", np.std(cv_scores))

        # Salvataggio del modello con input_example
        input_example = pd.DataFrame(X.iloc[:1])  # Usa una riga di X come esempio di input
        mlflow.sklearn.log_model(model, model_path, input_example=input_example)

        # Salvataggio classification report come JSON
        report_path = os.path.join(artifact_path, f"{model_name}_classification_report.json")
        with open(report_path, "w") as f:
            json.dump(reports, f, indent=4)

        # Log del classification report come artifact su MLflow
        mlflow.log_artifact(report_path)

    elapsed_time = time.time() - start_time
    print(f"[{model_name}] Training completato in {elapsed_time:.2f} secondi.")


In [21]:
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

def create_ann(input_dim, num_classes):
    model = Sequential([
        Dense(128, activation='relu', input_dim=input_dim),
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def train_and_evaluate_ann_mlflow(X, y, subjects, folds, experiment_name):
    print("\nInizio training modello: ANN")
    start_time = time.time()

    # Imposta l'esperimento in MLflow
    mlflow.set_experiment(experiment_name)

    with mlflow.start_run(run_name="ANN"):
        # Creazione delle directory per organizzare i file
        experiment_path = os.path.join("mlruns", experiment_name)
        model_path = os.path.join(experiment_path, "models", "ANN.h5")
        log_path = os.path.join(experiment_path, "logs", "ANN")
        artifact_path = os.path.join(experiment_path, "artifacts", "ANN")

        os.makedirs(model_path, exist_ok=True)
        os.makedirs(log_path, exist_ok=True)
        os.makedirs(artifact_path, exist_ok=True)

        # Log dei parametri dell'ANN
        mlflow.log_params({
            "Layers": "[128, 64, num_classes]",
            "Activation": "relu",
            "Optimizer": "adam",
            "Learning Rate": 0.01,
            "Loss": "sparse_categorical_crossentropy",
            "Epochs": 50,
            "Batch Size": 256,
            "Early Stopping": False
        })

        cv_scores = []
        reports = []
        training_logs = {}

        for fold_idx, (train_idx, test_idx) in enumerate(tqdm(folds, desc="Cross-validation folds")):
            model = create_ann(X.shape[1], len(np.unique(y)))

            # Imposta l'optimizer con learning rate 0.01
            model.compile(
                optimizer=Adam(learning_rate=0.01),  # Learning rate personalizzato
                loss="sparse_categorical_crossentropy",
                metrics=["accuracy"]
            )

            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            # Training senza Early Stopping
            history = model.fit(
                X_train, y_train,
                validation_data=(X_test, y_test),
                epochs=50, batch_size=256,
                verbose=1
            )

            # Salvataggio log di training
            training_logs[f"fold_{fold_idx}"] = {
                "loss": history.history["loss"],
                "accuracy": history.history["accuracy"],
                "val_loss": history.history["val_loss"],
                "val_accuracy": history.history["val_accuracy"]
            }

            # Valutazione
            y_pred = model.predict(X_test)
            y_pred_classes = np.argmax(y_pred, axis=1)
            score = accuracy_score(y_test, y_pred_classes)
            cv_scores.append(score)

            # Generiamo il classification report
            report = classification_report(y_test, y_pred_classes, output_dict=True)
            reports.append(report)

            # Creiamo e salviamo la Confusion Matrix
            cm_filename = os.path.join(log_path, f"confusion_matrix_fold_{fold_idx}.png")
            cm = confusion_matrix(y_test, y_pred_classes)
            plt.figure(figsize=(8, 6))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
            plt.xlabel('Predicted')
            plt.ylabel('Actual')
            plt.title(f'Confusion Matrix - ANN Fold {fold_idx}')
            plt.savefig(cm_filename)
            plt.close()

            # Log delle metriche su MLflow
            mlflow.log_metric(f"accuracy_fold_{fold_idx}", score)
            mlflow.log_artifact(cm_filename)  # Log della Confusion Matrix su MLflow

        # Log finali
        mlflow.log_metric("Mean_CV_Accuracy", np.mean(cv_scores))
        mlflow.log_metric("Std_CV_Accuracy", np.std(cv_scores))

        # Salvataggio modello ANN su MLflow con input_example
        input_example = tf.convert_to_tensor(X.iloc[:1].values.astype(np.float32))
        model.save(model_path)
        mlflow.tensorflow.log_model(model, model_path, input_example=input_example)

        # Salvataggio classification report come JSON
        report_path = os.path.join(artifact_path, "ANN_classification_report.json")
        with open(report_path, "w") as f:
            json.dump(reports, f, indent=4)
        mlflow.log_artifact(report_path)

        # Salvataggio dei log di training
        training_log_path = os.path.join(log_path, "training_logs.json")
        with open(training_log_path, "w") as f:
            json.dump(training_logs, f, indent=4)
        mlflow.log_artifact(training_log_path)

    elapsed_time = time.time() - start_time
    print(f"[ANN] Training completato in {elapsed_time:.2f} secondi.")


In [22]:
# Definisci il nome dell'esperimento
experiment_name = "Pipeline2.2"

print(f"Inizio training e logging con MLflow per l'esperimento '{experiment_name}'...")

# Allenamento per i modelli classici (Random Forest, XGBoost, Logistic Regression)
for model_name, model in models.items():
    train_and_evaluate_mlflow(model_name, model, X, y, subjects, folds, experiment_name)

# Allenamento per il modello ANN
train_and_evaluate_ann_mlflow(X, y, subjects, folds, experiment_name)

print(f"✅ Training completato per tutti i modelli! Controlla MLflow UI per i risultati dell'esperimento '{experiment_name}'.")


Inizio training e logging con MLflow per l'esperimento 'Pipeline2.2'...

Inizio training modello: Random Forest


Cross-validation folds: 100%|██████████| 5/5 [00:21<00:00,  4.39s/it]
/Users/giorgio/Desktop/SmartphonePositioning/tf-venv/lib/python3.11/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/849866340618810789/runs/3a5f8ff1a8ac4ffa99f1f5789c41f801
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/849866340618810789
[Random Forest] Training completato in 24.41 secondi.

Inizio training modello: XGBoost


Cross-validation folds: 100%|██████████| 5/5 [00:22<00:00,  4.45s/it]
/Users/giorgio/Desktop/SmartphonePositioning/tf-venv/lib/python3.11/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🏃 View run XGBoost at: http://127.0.0.1:5000/#/experiments/849866340618810789/runs/2c0a81111b1e49fe8b432662bdc0ecd9
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/849866340618810789
[XGBoost] Training completato in 23.80 secondi.

Inizio training modello: Logistic Regression


...

/Users/giorgio/Desktop/SmartphonePositioning/tf-venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.2500 - loss: 1928.5834 - val_accuracy: 0.2128 - val_loss: 1518.1959
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2411 - loss: 988.0105 - val_accuracy: 0.1935 - val_loss: 1094.0426
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2380 - loss: 1057.3955 - val_accuracy: 0.1900 - val_loss: 694.8416
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2311 - loss: 1000.6034 - val_accuracy: 0.2489 - val_loss: 779.7884
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2409 - loss: 970.9927 - val_accuracy: 0.1869 - val_loss: 1520.9258
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2351 - loss: 1181.1426 - val_accuracy: 0.2331 - val_loss: 1230.4032
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2429 - loss: 1033.5676 - val_accuracy: 0.2532 - val_loss: 820.1736
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2321 - loss:

/Users/giorgio/Desktop/SmartphonePositioning/tf-venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/giorgio/Desktop/SmartphonePositioning/tf-venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/giorgio/Desktop/SmartphonePositioning/tf-venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

🏃 View run ANN at: http://127.0.0.1:5000/#/experiments/849866340618810789/runs/5f7d8d61ff4a4e43b9664296db3fa8d2
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/849866340618810789


ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=mlruns/Pipeline2.2/models/ANN.